In [ ]:
# Insert into medium

# Create new Index
curl -XDELETE 'localhost:9200/htwberlin?pretty'
curl -XPUT 'localhost:9200/htwberlin?pretty'

# Erzeugen eines JSON-Konformen Datenstroms (hehe)
cat docs.json | jq -c '{ index: { } },. ' > bulk.json

#alternative mit Index + Type
cat docs.json | jq -c '{ index: { _index: "htwberlin", _type: "medium" } },. '

# Wer kein Bash/jq hat sollte einfach den Header in die Datei reinkopieren können
Header = '{ index: { } }' # <-- Muss über jedem eintrag stehen.

# Sieht am Ende so aus:

In [ ]:
{"index":{}}
{"owner":"3","viewable":[3,2],"public":true,"filename":"Bachelor Party 2015.jpg"}
{"index":{}}
{"owner":"2","viewable":[3,2,1],"public":true,"filename":"Bachelor Party 2016.jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":true,"filename":"Bachelor Party 2017.jpg"}
{"index":{}}
{"owner":"3","viewable":[3,2],"public":true,"filename":"Master Party 2015.jpg"}
{"index":{}}
{"owner":"2","viewable":[3,2,1],"public":true,"filename":"Master Party 2016.jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":true,"filename":"Master Party 2017.jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":false,"filename":"Master Party 2017 (geheim).jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":true,"filename":"Master Absolventen 2017.jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":true,"filename":"Dozenten.jpg"}
{"index":{}}
{"owner":"1","viewable":[3,2,1],"public":true,"filename":"HTW Gebaeude C.jpg"}

In [ ]:
# Alternativ alles Einzeln via XPOST einfügen:
curl -XPOST 'localhost:9200/htwberlin/medium/?pretty' -H 'Content-Type: application/json' -d'
{"owner":"3","viewable":[3,2],"public":true,"filename":"Bachelor Party 2015.jpg"}'

In [ ]:
# Übergebe die Daten via dem _bulk entpunkt an ElasticSearch
# Wichtig: Wenn ihr den Index/type angebt darf der Header leer sein
# Ansonsten muss dort _index : index und _type : type angegeben werden.
curl -XPOST 'localhost:9200/htwberlin/medium/_bulk?pretty' --data-binary @bulk.json

# Verify (Sollten 10 Hits sein)
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty'

In [ ]:
# 1) Finde Dokumente welche 'party' oder 'absolventen' im Dateinamen haben.
# Match Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
"query": {
  "match": { "filename" : "party absolventen" }
  }
}'
# Hier ist durch match ein richtiges Scoring

# Alternativ mit einem Constant_Score/Terms Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "constant_score" : {
            "filter" : {
                "terms" : { "filename" : ["party", "absolventen"]}
            }
        }
    }
}'
# Hier wird nur auf alle ergebnisse gefiltert

In [ ]:
# 2) Ich muss das Dokument sehen dürfen.
# Simple Match Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
"query": {
  "match": { "viewable" : "1" }
  }
}'

# Harder Bool-Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool" : {
      "must" : {
        "term" : { "viewable" : "1" }
      }
    }
  }
}'

In [ ]:
# 3) Es dürfen nur öffentliche Dokumente gefunden werden.
# Match Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
"query": {
  "match": { "public" : "true" }
  }
}'

# Bool Query
curl -XPOST 'localhost:9200/htwberlin/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool" : {
      "must" : {
        "term" : { "public" : "true" }
      }
    }
  }
}'

In [ ]:
# 4) Dokumente, von denen ich der Eigentürmer bin, sollen in der Ergebnismenge stärker berücksichtigt werden
# (eine höhere Relevanz haben/weiter oben erscheinen).
curl -XPOST 'localhost:9200/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "should": {
        "term": {
          "owner": {
            "value": "1",
            "boost": 2
          }
        }
      }
    }
  }
}
'

In [ ]:
# One Query to rule them all
curl -XPOST 'localhost:9200/_search?pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool": {
      "must" : [
        {"term" : { "public" : "true" }},
        {"term" : { "viewable" : "1" }},
        {"match" : { "filename" : "party absolventen"}}
        ],
      "should": {
        "term": {
          "owner": {
            "value": "1"
          }
        }
      }
    }
  }
}
'

# public: true und viewable:1 sind eher als filter zu implementieren (simples ja/nein verhalten)